# Querying Data from `FTA Bus Award StoryMap`

Currently FTA uses ArcGIS storymap to display their bus award data. Is is possible to extract this data using their REST server.
Extracting bus award data from a server will be the most reliable way to see the data.

In [1]:
import geopandas as gpd
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

In [2]:
from calitp_data_analysis.sql import to_snakecase

# Get geojson from ArcGIS Rest Server 

https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query
https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query


# Steps to Query Rest Server

1. Set `Where:` field to `1=1`
2. Set `Out Field:` field to `*`
3. Set `Format:` to `Geojson`
4. Click `Query(GET)`
5. Copy paste given URL 

In [3]:
fy24 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

fy23 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=" 


fy_24_data = to_snakecase(gpd.read_file(fy24))
fy_23_data = to_snakecase(gpd.read_file(fy23))

display(
    fy_24_data.shape,
    type(fy_24_data),
    fy_24_data.info(),
    fy_23_data.shape,
    type(fy_23_data),
    fy_23_data.info(),
)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   application_number_       117 non-null    object  
 1   agency_name               117 non-null    object  
 2   low_no_bbf                117 non-null    object  
 3   project_description       117 non-null    object  
 4   number_of_buses_          117 non-null    int64   
 5   funding_amount            116 non-null    float64 
 6   project_title             117 non-null    object  
 7   project_type              117 non-null    object  
 8   display_type              117 non-null    object  
 9   overall_propulsion_type   117 non-null    object  
 10  specific_propulsion_type  117 non-null    object  
 11  areas_served              117 non-null    object  
 12  congressional_districts   117 non-null    object  
 13  address_line_1            117 non-null    

(117, 22)

geopandas.geodataframe.GeoDataFrame

None

(130, 25)

geopandas.geodataframe.GeoDataFrame

None

# Explore FY23 data

In [11]:
col_list =[
    "application_number",
    "project_sponsor",
    "project_description",
    "original_statement_of_work",
    "project_type",
    "funding",
    "number_of_buses",
    "traditional_buses",
    "low_emission_buses",
    "zero_emission_buses",
    "propulsion_type"
]

fy23_zeb_bus = fy_23_data[(fy_23_data["traditional_buses"] == 0) &
          (fy_23_data["low_emission_buses"] == 0) &
          (fy_23_data["zero_emission_buses"] > 0)][col_list]

In [5]:
display(fy23_zeb_bus.shape,
        fy23_zeb_bus.columns
       )

(42, 9)

Index(['application_number', 'project_sponsor', 'project_description',
       'funding', 'number_of_buses', 'traditional_buses', 'low_emission_buses',
       'zero_emission_buses', 'propulsion_type'],
      dtype='object')

In [8]:
# looks pretty good. will have to use some of the functions identify specific prop type and if projects include non-bus items
fy23_zeb_bus

,application_number,project_sponsor,project_description,original_statement_of_work,funding,number_of_buses,traditional_buses,low_emission_buses,zero_emission_buses,propulsion_type
4,BUSFAC-2023-3745,Solano County Transit,SolTrans will receive funding to buy battery-electric buses and charging equipment and develop a workforce development training program as it plans to convert its entire fixed-route fleet to zero emissions by 2026. The project will improve air quality as well as service reliability and improve transit service for residents in the Solano County cities of Vallejo and Benicia.,"SolTrans seeks funding for battery-electric buses and required charging equipment that will convert our entire local fixed route fleet to 100% zero emissions by 2026. SolTrans is partnering with CTE, GILLIG, Chargepoint, IINDUCTEV, WSP, and PG&E on this project .BEBs: SolTrans is requesting funding to purchase 14 GILLIG battery electric plug-in buses with inductive charging capability to replace 2011 diesel-hybrid buses will reach the end of their useful life of 14 years by the time their replacements arrive, if funded.Plug-in Chargers: SolTrans requests funding for eleven Chargepoint plug-in chargers to be installed at our operations and maintenance bus yard to charge buses set for delivery in September 2024 and the additional 14 buses we plan to procure if awarded this grant. Inductive Chargers: SolTrans also requests funding for three INDUCTEV inductive chargers to be installed at the Vallejo Transit Center to provide en-route charging to ensure continuity of service.","$12,458,500",14 (BEB),0,0,14,Zero
5,BUSFAC-2023-3759,Oregon Department of Transportation on behalf of Hood River County Transportation District,"The Oregon Department of Transportation on behalf of the Hood River County Transportation District's Columbia Area Transit will receive funding to buy buses and charging equipment, upgrade its vehicle storage area, and develop a workforce training program. The project will reduce vehicle emissions, improve transit service and reliability, and support job development for residents living in Hood River County.","CAT working together with the vehicle manufacture, Proterra and our the local utility provider, Pacificorp will deploy two (2) 40 EVs, two (2) 35 EV, one (1) EV Cutaway and one (1) EV van into our 19 vehicle fleet. Fleet deployment will include upgrades to our vehicle storage area, installation of pedestal and depot chargers and the implementation of a well rounded workforce development training program for our employees.","$6,424,808",6,0,0,6,Zero
6,BUSFAC-2023-3761,City Of Tallahassee,"The city of Tallahassee's StarMetro system will receive funding to buy battery-electric buses, support charging infrastructure and develop a workforce training program for its mechanics and operators to maintain the fleet. The project will help reduce emissions and improve service and reliability for riders in and around Tallahassee.","To reach the City of Tallahassee's goal for 100% Battery Electric Bus transportation by 2035, StarMetro must purchase eight BEBs in FY23 and start building Phase One of the electric charging infrastructure necessary to support a full BEB fleet. Operating BEBs since 2012, StarMetro is a statewide pioneer in BEB deployment and continues to partner with Proterra and GILLIG for additional BEB support on bus purchases, infrastructure projects, and route optimization. If awarded facility funding, StarMetro plans to start Phase One of its electric charging infrastructure facility immediately to accommodate additional BEBs as they arrive. Overnight charging will enable route restructuring to serve disadvantaged communities with the new emission-free vehicles. Low-No funding will also assist in workforce development efforts with local partners, as StarMetro continues to focus on training and maintaining BEB mechanics and operators able to serve an entire fleet.","$20,370,793",8,0,0,8,Zero
15,BUSFAC-2023-3899,Santa Cr

# Explore FY 24 data

In [9]:
list(fy_24_data.columns)

['application_number_',
 'agency_name',
 'low_no_bbf',
 'project_description',
 'number_of_buses_',
 'funding_amount',
 'project_title',
 'project_type',
 'display_type',
 'overall_propulsion_type',
 'specific_propulsion_type',
 'areas_served',
 'congressional_districts',
 'address_line_1',
 'city',
 'state',
 'additional_zip',
 'zip_code',
 'x',
 'y',
 'fid2',
 'geometry']

In [12]:
col_list_24 = [
    "application_number_",
    "agency_name",
    "project_description",
    "project_type",
    "funding_amount",
    "number_of_buses_",
    "overall_propulsion_type",
    "specific_propulsion_type"
]

fy_24_data[col_list_24].head()

,application_number_,agency_name,project_description,project_type,funding_amount,number_of_buses_,overall_propulsion_type,specific_propulsion_type
0,BUSFAC-2024-4750,"Alaska Department of Transportation, on behalf of City and Borough of Juneau, Capital Transit","Alaska Department of Transportation, on behalf of Capital Transit, will receive funding to buy new electric buses, install charging equipment, and initiate a workforce development program. The project will improve service, reliability and air quality throughout the city and borough of Juneau in Alaska’s Southeast panhandle.",Vehicle | Facility | Chargers,11855112.0,6,Zero,Battery electric
1,BUSFAC-2024-5061,City of Montgomery / The M Public Transit System,"The city of Montgomery's The M transit system will receive funding to buy battery electric buses, charging equipment and to initiate a workforce training program. The project will ensure continued service reliability, maintain a state of good repair and, by improving air quality, advance environmental justice in the Montgomery community.",Vehicle | Chargers,16941377.0,8,Zero,Battery electric
2,LOWNO-2024-4964,Rock Region Metropolitan Transit Authority,"The Rock Region Metropolitan Transit Authority will receive funding to buy low-emission (compressed natural gas) buses to replace older diesel buses and a simulator to train staff in the new technology. The project will help improve service, reliability and air quality in Little Rock and North Little Rock.",Vehicle | Equipment,3149667.0,5,Low,CNG
3,BUSFAC-2024-4967,"City of Tucson, Sun Tran / Sun Van","The city of Tucson and Sun Tran will receive funding to replace aging equipment at its Northwest Bus Maintenance Facility and upgrade multiple bus stops. By installing dozens of shelters, adding trees for shade and improving signage through Braille, large print, wayfinding and audible information, the agency will make their system more accessible and equitable.",Facility,11385600.0,0,N/A,N/A
4,BUSFAC-2024-4986,Salt River Pima-Maricopa Indian Community (SRPMIC),"The Salt River Pima-Maricopa Indian Community will receive funding to buy accessible vans to replace vehicles nearing the end of their useful life. The vans will provide on-demand service for residents to reach jobs, school, recreation and other essential services throughout Scottsdale, Mesa and Tempe.",Vehicle,425001.0,3,Standard,Gasoline


In [13]:
fy_24_data["specific_propulsion_type"].value_counts()

Battery electric                                            38
Diesel-electric hybrid                                      14
N/A                                                         12
CNG                                                         11
Gasoline                                                     9
Hydrogen fuel cell                                           8
Diesel                                                       6
Propane                                                      4
Diesel | Gasoline                                            3
Battery electric | Hydrogen fuel cell                        2
Diesel-electric hybrid | Hydrogen fuel cell                  2
Battery electric | Diesel-electric hybrid | Propane          1
CNG | Gasoline                                               1
CNG | Diesel | Diesel-electric hybrid | Gasoline | Other     1
CNG | Diesel-electric hybrid                                 1
Battery electric | Propane                             